<a href="https://colab.research.google.com/github/mkane968/Webscraping-Wikipedia-Tables/blob/main/Webscraping_Wikipedia_Tables_from_Page_Names_List.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Download packages

In [ ]:
#Import libraries
!pip install wikipedia
import wikipedia
import urllib
from bs4 import BeautifulSoup as bsoup
import bleach
import pandas as pd
import requests
from bs4 import BeautifulSoup

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=dff966a0ff211c09f81504d49dace181d639c1c3d0d8137c1880d9b2c17b5133
  Stored in directory: /root/.cache/pip/wheels/15/93/6d/5b2c68b8a64c7a7a04947b4ed6d89fb557dcc6bc27d1d7f3ba
Successfully built wikipedia


##Import names of all pages to be scraped (CSV file) and search wikipedia for each name

In [ ]:
#Import csv with list of url links
from google.colab import files
uploaded = files.upload()


Saving Companies.csv to Companies.csv


In [ ]:
#Make search terms into dataframe
import pandas as pd
df = pd.read_csv('Companies.csv')  

link_list = df['All_Companies'].to_list()
link_list

In [ ]:
#Search wikipedia for each term and append first result to a new list
search_results = []
for item in link_list:   
  result = wikipedia.search(item, results = 1)
  search_results.append(result)

#Check search results
len(search_results)

444

In [ ]:
#Iterate through list of search results, get url of each page and append to list
page_urls = []
#Store list of companies where no URL was found for first search result
no_urls = []

for item in search_results:
    try:
      url = wikipedia.page(item, auto_suggest=False).url
      page_urls.append(url)
    except:
      no_urls.append(item)


In [ ]:
page_urls

['https://en.wikipedia.org/wiki/2015',
 'https://en.wikipedia.org/wiki/2K_(company)',
 'https://en.wikipedia.org/wiki/2K_(company)',
 'https://en.wikipedia.org/wiki/Radical_Entertainment',
 'https://en.wikipedia.org/wiki/3DO_Interactive_Multiplayer',
 'https://en.wikipedia.org/wiki/The_Invincibles_(English_football)',
 'https://en.wikipedia.org/wiki/Bombardier_Global_Express',
 'https://en.wikipedia.org/wiki/Studio_7',
 'https://en.wikipedia.org/wiki/Acclaim_Entertainment',
 'https://en.wikipedia.org/wiki/List_of_Acclaim_Entertainment_subsidiaries',
 'https://en.wikipedia.org/wiki/List_of_Acclaim_Entertainment_subsidiaries',
 'https://en.wikipedia.org/wiki/Acclaim_Studios_Austin',
 'https://en.wikipedia.org/wiki/List_of_Acclaim_Entertainment_subsidiaries',
 'https://en.wikipedia.org/wiki/Acclaim_Studios_Manchester',
 'https://en.wikipedia.org/wiki/List_of_Acclaim_Entertainment_subsidiaries',
 'https://en.wikipedia.org/wiki/Activision',
 'https://en.wikipedia.org/wiki/Activision',
 'htt

In [ ]:
#Get # of page urls that were found
print(len(page_urls))

#Get # of pages that were not found
print(len(no_urls))

425
19


##Use BeautifulSoup to scrape pages from list of retrieved urls

In [ ]:
#Create empty list for soup objects and titles
soups = []
all_scraped_title_list = []
all_scraped_title_strings = []

# create function to harvest data from each URL
def get_data(page_url):
      page = requests.get(page_url)
      soup = BeautifulSoup(page.text, 'html.parser')
      #Get title of page
      title = soup.find(id="firstHeading")
      all_scraped_title_list.append(title)
      #Append scraped data to list
      soups.append(soup)
  

# call the function for each URL in the list
for url in page_urls:
    get_data(url) 

#Get strings for all pages with titles
for title in all_scraped_title_list: 
    string = title.string
    #Turn into string
    string = str(string)
    all_scraped_title_strings.append(string)

#Confirm # of pages scraped (should be same as page_urls)
len(all_scraped_title_strings)

425

##Confirm how many pages have the table we're looking for (infobox vcard)

In [ ]:
titles_with_tables = []
all_columns = []

#Find table with headquarters values (use inspect to find on wiki pages)
for soup in soups: 
  title = soup.find(id="firstHeading")
  tables = soup.find_all('table',class_='infobox vcard')
  #Get names of all columns in table and append to list
  for table in tables:
    column_names = [item.get_text() for item in tables[0].find_all('th')]
    all_columns.append(column_names)
    #Get names of all titles with the right table and append to list
    titles_with_tables.append(title)


In [ ]:
#Get list of strings for all the pages with the correct table
table_title_strings = []
for title in titles_with_tables: 
    string = title.string
    #Turn into string
    string = str(string)
    table_title_strings.append(string)
len(table_title_strings)

315

In [ ]:
#Get all titles where right table was NOT found and append to list for later
from collections import Counter
no_table_strings = list((Counter(all_scraped_title_strings)-Counter(table_title_strings)).elements())

##Confirm how many pages have the value we're looking for (headquarters)

In [ ]:
#Associate all titles where table was found with table column values
keys = table_title_strings
values = all_columns
res = {keys[i]: values[i] for i in range(len(keys))}

#Create dataframe based on title and table column values
test = pd.DataFrame.from_dict(res, orient='index')

In [ ]:
#Iterate through dataframe and all company rows that include headquarters as column value 
df = test[test.apply(lambda x: x.str.contains('Headquarters')).any(axis=1)]
df = df.reset_index()
df_companies = df['index']

#Append matching company names to list
titles_with_hq = df_companies.tolist()

In [ ]:
# Get all titles where headquarters was NOT included as row in table and append to list for later
from collections import Counter
no_hq_strings = list((Counter(table_title_strings)-Counter(titles_with_hq)).elements())

In [ ]:
len(titles_with_hq)

256

In [ ]:
len(no_hq_strings)

59

#Webscrape only URLS of pages where table and HQ column are present

In [ ]:
#Search wikipedia for each term and append first result to a new list
search_results2 = []
for item in titles_with_hq:
  result = wikipedia.search(item, results = 1)
  search_results2.append(result)

#Check search results
search_results2

#Iterate through list of search results, get url of each page and append to list
page_urls2 = []
no_urls2 = []

for item in search_results2:
    try:
      url = wikipedia.page(item, auto_suggest=False).url
      page_urls2.append(url)
    except:
      no_urls2.append(item)

page_urls2


In [ ]:
#Get # of page urls that were found
print(len(page_urls2))

#Get # of pages that were not found (should be 0)
print(len(no_urls2))

256
0


In [ ]:
#Use list of urls established above
#Set up lists to append scraped data
soups2 = []
all_scraped_title_list2 = []

# create function to harvest data from each URL
def get_data2(page_url):
      page = requests.get(page_url)
      soup = BeautifulSoup(page.text, 'html.parser')
      #Get title of page
      title = soup.find(id="firstHeading")
      all_scraped_title_list2.append(title)
      #Append scraped data to list
      soups2.append(soup)
  

 # call the function for each URL in the list
for url in page_urls2:
    get_data2(url) 

In [ ]:
#Get correct table for all titles and append table to list, title to list
titles_with_tables2 = []
right_tables = []
for soup in soups2: 
  title = soup.find(id="firstHeading")
  tables = soup.find_all('table',class_='infobox vcard')
  for table in tables:
    titles_with_tables2.append(title)
    right_tables.append(table)
  
#Append all titles from pages with scraped tables to list
title_strings2 = []
for title in titles_with_tables2: 
    string = title.string
    #Turn into string
    string = str(string)
    title_strings2.append(string)
title_strings2  

## Create dataframe from tables of all pages and clean for HQ values only

In [ ]:
#Append values from each correct table to dataframe
scraped_tables_df = pd.DataFrame()

#Convert table into dataframe
for table in right_tables:
  scraped_tables_df = scraped_tables_df.append(pd.read_html(str(table)))

In [ ]:
# Clean datafarme (transpose, add column headers)
scraped_tables_df = scraped_tables_df.T
scraped_tables_df.columns = scraped_tables_df.iloc[0]
scraped_tables_df = scraped_tables_df.drop(scraped_tables_df.index[0])
scraped_tables_df

,Logo since 2021,Type,Industry,Founded,Headquarters,Key people,Products,Parent,Subsidiaries,Website,...,Products,Website,Type,Industry,Founded,Founders,Defunct,Headquarters,Products,Parent
1,Logo since 2021,Subsidiary,Video games,"January 25, 2005; 17 years agoNew York City, US","Novato, California, US",David Ismailer (president)Phil Dixon (COO)Meli...,BioShock series Borderlands series Civilizatio...,Take-Two Interactive,See § Studios,2k.com,...,Zork NemesisSpec Ops seriesSaw seriesBlackligh...,zombie.com (archived),Subsidiary,Video games,"July 25, 1991; 31 years agoCosta Mesa, Califor...",Ed ZobristWilliam Novak,December 2007,"El Segundo, California, U.S.",Metal FatigueMr. Bones,MumboJumbo (2005–2007)


In [ ]:
#Keep only headquarters values
scraped_tables_df = scraped_tables_df['Headquarters']

#Transpose
scraped_tables_df = scraped_tables_df.T
scraped_tables_df

,1,Company
0,,
Headquarters,"Novato, California, US",2K (company)
Headquarters,"Vancouver, British Columbia, Canada",Radical Entertainment
Headquarters,"Glen Cove, New York, U.S.",Acclaim Entertainment
Headquarters,"Austin, Texas, U.S.",Acclaim Studios Austin
Headquarters,"Manchester, England",Acclaim Studios Manchester
...,...,...
Headquarters,"Taipei, Republic of China",XPEC Entertainment
Headquarters,"New York City (prior), Fairfield, Connecticut ...",XS Games
Headquarters,"Berlin, Germany",Yager Development


## Associate HQ columns with company names, conduct final cleaning, and download to CSV

In [ ]:
#Add company titles as new column
scraped_tables_df['Company'] = title_strings2
scraped_tables_df

In [ ]:
#Clean dataframe
scraped_tables_df = scraped_tables_df.reset_index(drop=True)
cols = scraped_tables_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
scraped_tables_df = scraped_tables_df[cols]
scraped_tables_df.rename(columns={1: 'Headquarters'}, inplace=True)


#Print finalized dataframe
scraped_tables_df

,Company,Headquarters
0,2K (company),"Novato, California, US"
1,Radical Entertainment,"Vancouver, British Columbia, Canada"
2,Acclaim Entertainment,"Glen Cove, New York, U.S."
3,Acclaim Studios Austin,"Austin, Texas, U.S."
4,Acclaim Studios Manchester,"Manchester, England"
...,...,...
255,XPEC Entertainment,"Taipei, Republic of China"
256,XS Games,"New York City (prior), Fairfield, Connecticut ..."
257,Yager Development,"Berlin, Germany"
258,Zombie Studios,"Seattle, Washington, United States[1]"


In [ ]:
#Export to csv
from google.colab import files
scraped_tables_df.to_csv('headquarters_output.csv', encoding = 'utf-8-sig') 
files.download('headquarters_output.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

#Compile companies without output for further investigation

In [ ]:
#List of companies where no page URL was found 
#Possible fix - first URL was ambiguous, rerun with different # search result
no_urls

#List of companies where no table was found
#Possible fix - searching wrong URL, rerun with different # search result
no_table_strings

#List of companies where headquarters was not in tables
#Possible fix - look @ tables by hand to determine if correct page; if so, enter HQ manually/from different site
no_hq_strings


#Append all into dataframe and clean
no_results = [no_urls, no_table_strings, no_hq_strings]
no_results_df=pd.DataFrame(no_results)
no_results = no_results_df.T
no_results = no_results.rename(columns={0: "First_Search_Result_Not_Scraped", 1: "No_Infobox_VCard_Table", 2: "No_HQ_Column"}, errors="raise")

#Download results
no_results.to_csv('no_results.csv', encoding = 'utf-8-sig') 
files.download('no_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
#Possible fixes for pages with wrong URLs
#Re-search and append (if correct page is not first result)
wikipedia.search("Zed Two", results = 4)
search_results.append("result")